## IterationManagers

**Spencer Lyon**

*Date: 4-23-15* n   

## Intro

Many economics problems feature iterative algorithms:

- Value function iteration (fixed point iteration)
- Optimization/root finding
- Estimation: MCMC, Generalized Method of Moments (see below)


### Example: Newton's method

Most of these tasks share a common structure. 

To see the structure, consider this simple implementation of newton's method to find the maxima of a function (or, equivalently the root of its derivative):

In [1]:
function newton(fp::Function, fpp::Function, init; tol::Float64=1e-12,
                 maxiter::Int=500, verbose::Bool=true, print_skip::Int=5)
    # Stage 1: Setup
    x = copy(init)
    dist = 1.
    iter = 0
    elapsed = 0.0
    old_time = time()

    while dist > tol && iter < maxiter
        # Stage 2: Iteration
        x_new = x - fpp(x) \ fp(x) 

        # Stage 3: Between iteration processing
        dist = maxabs(x - x_new)
        iter += 1
        new_time = time()
        elapsed += new_time - old_time

        if verbose && iter % print_skip == 0
            println("Iteration: $iter\t dist: $(round(dist, 4))\t elapsed: $(elapsed)")
        end

        x = copy(x_new)
        old_time = new_time
    end

    # Stage 4: post iteration processing
    if verbose
        if iter >= maxiter
            warn("failed to converge in $maxiter iterations")
        else
            println("Converged successful ly after $iter iterations")
        end
    end
    x
end

newton (generic function with 1 method)

The core part of this algorithm is on line 12 where we compute `x_new - fpp(x) \ fp(x)`. 

But, what the code actually does is not important to our discussion; we just care about its structure. Notice that there are 4 main sections (stages) of the code:

1. Setup
2. Iteration
3. Between iteration processing
4. Post iteration processing

Almost all the iterative algorithms I have ever written have either this exact structure, or a subset of it (not all algorithms need a post-iteration step, for example). 

Given that this structure is so common, we should be able to automate it and remove some boiler-plate. Well, it turns out that we can! 

Consider another version of the `newton` function from above:

In [2]:
using IterationManagers

function newton2(fp::Function, fpp::Function, init; tol::Float64=1e-12,
                 maxiter::Int=500, verbose::Bool=true, print_skip::Int=5)
    # setup manager and state
    mgr = DefaultManager(tol, maxiter, verbose, print_skip)
    istate = DefaultState(init)
    
    # define function to produce next iterate  
    newton_step(x) =  x - fpp(x) \ fp(x)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    # stages 2, 3, 4 in one shot!
    managed_iteration(newton_step, mgr, istate)
end

newton2 (generic function with 1 method)

In this function we take the same arguments and use them to construct an instance of `DefaultManager` and `DefaultState`. 

We then write a function to take the current "state" of the iterations (`x`) and produce the next iterate -- this is the `newton_step` function.

We then call the `managed_iteration` function, which has the following signature:

```julia
managed_iteration(f::Function, mgr::IterationManager, istate::IterationState; by::Function=default_by)
```

The `managed_iteration` function will do all the same pre-mid- and post-processing that we did by hand in the original version of `newton`.

I wish to point out that we could also write a one-line version of newton's method using a special keyword argument version of `managed_iteration` that constructs the manager and state automatically and by leveraging Julia's [do block syntax](http://julia.readthedocs.org/en/latest/manual/functions/#do-block-syntax-for-function-arguments):

I wish to point out that we could have impleneted a one-line version this function using a special method of `managed_iteration` that constructs the `IterationManager` and `DefaultState` for us and passing an anonymous function as the first argument

In [3]:
function newton3(fp::Function, fpp::Function, init; tol::Float64=1e-12,
                 maxiter::Int=500, verbose::Bool=true, print_skip::Int=5)
    # all four stages in one!
    managed_iteration(x->x-fpp(x)\fp(x), init; tol=tol, maxiter=maxiter, 
                      print_skip=print_skip, verbose=verbose)
end

newton3 (generic function with 1 method)

Let's do a quick test of our functions and then see how IterationManagers is implemented

In [4]:
# let's test this out
rosenbrock(x::Vector) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2

function rosenbrock_gradient(x::Vector)
    out = similar(x)
    out[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    out[2] = 200.0 * (x[2] - x[1]^2)
    out
end

function rosenbrock_hessian(x::Vector)
    out = Array(Float64, length(x), length(x))
    out[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    out[1, 2] = -400.0 * x[1]
    out[2, 1] = -400.0 * x[1]
    out[2, 2] = 200.0
    return out
end
const f = rosenbrock
const fp = rosenbrock_gradient
const fpp = rosenbrock_hessian

rosenbrock_hessian (generic function with 1 method)

In [5]:
opt_x = newton(fp, fpp, fill(50.0, 2); print_skip=1)

Iteration: 1	 dist: 2449.99	 elapsed: 1.2602849006652832
Iteration: 2	 dist: 4899.9704	 elapsed: 1.6795709133148193
Iteration: 3	 dist: 2400.9806	 elapsed: 1.679621934890747
Iteration: 4	 dist: 0.0002	 elapsed: 1.67964506149292
Iteration: 5	 dist: 0.0	 elapsed: 1.6796679496765137
Iteration: 6	 dist: 0.0	 elapsed: 1.6796901226043701
Converged successfully after 6 iterations


2-element Array{Float64,1}:
 1.0
 1.0

In [6]:
f(opt_x)

0.0

In [7]:
opt_x2 = newton2(fp, fpp, fill(50.0, 2); print_skip=1).prev
opt_x3 = newton 3(fp, fpp, fill(50.0, 2); print_skip=1).prev

# check that we did indeed get the same answer
map(x->maxabs(opt_x - x) <= eps(), (opt_x2, opt_x3))

Iteration    Distance       Elapsed (seconds)
---------------------------------------------
1            2.44999e+03    0.35528           
2            4.89997e+03    0.50673           
3            2.40098e+03    0.50710           
4            1.95988e-04    0.50744           
5            9.60096e-09    0.50778           
6            0.00000e+00    0.50811           
Iteration    Distance       Elapsed (seconds)
---------------------------------------------
1            2.44999e+03    0.01630           
2            4.89997e+03    0.01672           
3            2.40098e+03    0.01708           
4            1.95988e-04    0.01744           
5            9.60096e-09    0.01779           
6            0.00000e+00    0.01814           


(true,true)

### Implementation

Live-demo, or see [package repo on github](https://github.com/spencerlyon2/IterationManagers.jl/tree/master)

### Example: GMM

Generalized method of moments (GMM) is a common and powerful econometric technique for estimating the parameters of a statistical model.

It does not require distributional assumptions, only that certain moment conditions be satisfied by the parameter estimates.

Moment conditions can come from anywhere, but in economic models they are often given by first order necessary conditions for some optimizing agent.

#### Basic theory

Suppose you have $T$ observations of data $\left\{Y_t \right\}_{t=1}^T$, where $\forall t, Y_t \in \mathbb{R}^n$.

Assume the data is generated by a statistical model, indexed by a vector of parameters $\theta \in \Theta \subset \mathbb{R}^k$.

Let $g: \mathbb{R}^n \times \mathbb{R}^k \rightarrow \mathbb{R}$ be a function of moment conditions, such that 

$$m(\theta^*) \equiv E \left[g(Y_t, \theta^*) \right] = 0$$

Given the data $\left\{Y_t \right\}_{t=1}^T$ and a potential parameter vector $\theta$, we can construct sample moments:

$$\hat{m}(\theta) \equiv \frac{1}{T} \sum_{t=1}^T g(Y_t, \theta)$$

We choose an optimal estimate of the parametes vector, $\hat{\theta}$, that minimizes the GMM criterion:


$$\hat{\theta} \equiv \text{argmin}_{\theta \in \Theta} \hat{m}(\theta)' W \hat{m}(\theta),$$

where $W$ is a positive definite weighting matrix.

#### Algorithm

There are many variants of algorithms used to produce estimates $\hat{\theta}$.

Three of them are the most common:

(1)   One-step GMM: use $W = I$ and minimize the GMM criterion

(2) Two-step GMM: use $W = I$ to obtain $\hat{\theta}_1$, then set $W_1 = H(\theta_1) \equiv \text{Cov}(\hat{\theta}_1)^{-1}$ to obtain final $\hat{\theta}$ ($W_1$ has been proven to be the "optimal" $W$ in terms of efficiency)

(3) Iterative GMM: Fix a small tolerance level $\epsilon$, choose initial guess $\theta_0$, , let $i=1$,and set $W_0 = I$. At step $i$

1. Use $W_{i-1}$ to obtain $\hat{\theta}_i$
2. Compute then $W_i = H(\hat{\theta}_i)$ 
3. Check if $||\hat{\theta}_i - \hat{\theta}_{i-1}||<\epsilon$. if so, stop $\hat{\theta}_i$ is solution, otherwise return to step A

#### Implementation

We can leverage `IterationManagers` to write a single version of a `gmm` routine, where the algorithm that is used dependso on which `T <: IterationManger` is used.

##### Managers

First, let's implement the three managers (note each has a field `k` specifying how the covariance matrix should be computed within the function $H$ above)

```julia
using IterationManagers
import CovarianceMatrices.RobustVariance

immutable OneStepGMM <: IterationManager
    k::RobustVariance
end

OneStepGMM(k::RobustVariance=HC0()) = OneStepGMM(k)

immutable TwoStepGMM <: IterationManager
    k::RobustVariance
end

TwoStepGMM(k::RobustVariance=HC0()) = TwoStepGMM(k)

immutable IterativeGMM <: IterationManager
    k::RobustVariance
    tol::Float64
    maxiter::Int
end

function IterativeGMM(k::RobustVariance=HC0(), tol::Float64=1e-12,
                       maxiter::Int=500)
    IterativeGMM(k, tol, maxiter)
end

# note, we can use the DefaultState here
finished(::OneStepGMM, ist::DefaultState) = ist.n >= 1
finished(::TwoStepGMM, ist::DefaultState) = ist.n >= 2
function finished(mgr::IterativeGMM, ist::DefaultState)
    ist.n > mgr.maxiter || abs(ist.change) <= mgr.tol
end
```

##### Core algorithm

Suppose we have the following functions defined:

- `moment_func::Function` is our moment function $g$ from above
- `optimal_w(mf::Function, θ::Vector, k::RobustCovariance)` is $H$ from above
- `minimize_crit(mf::Function, W::Matrix)` gets $\hat{\theta}$, given $g$, $W$

(NOTE: These functions are defined as closures in the main `gmm` routine to pass the data ($Y_t$) to `moment_func` just once)

Now, the core iterative part of algorithm can be written as follows:

```julia
# mgr and istate either constructed above or passed as function arguments
managed_iteration(mgr, istate) do θ
    W = optimal_W(moment_func, θ, istate.k)
    minimize_crit(momoent_func, W)  # new theta
end
```

#### Comments

Using this functional style combined with multiple dispatch allows us focus on the actual algoirthm, instead of worrying about "accounting" deatails like managing the state (keeping track of $\theta_i$ and $\theta_{i-1}$) or updating the user (printing status updates).

This not only made the code shorter (easier to maintain), but helped us think in a more modular way and focus on what makes this specific problem different than other iterative algorithms